In [1]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
import itertools as it
from prettytable import PrettyTable
# from nanrms import *
# from handover import *
# from mask import *
import matplotlib.pylab as plt
import seaborn as sns

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import *

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度
# plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def set_data(df, mode='pcap', tz=0):
    if mode == 'pcap':
        df['seq'] = df['seq'].astype('Int32')
        # df['rpkg'] = df['rpkg'].astype('Int8')
        df['frame_id'] = df['frame_id'].astype('Int32')
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['xmit_time'] = pd.to_datetime(df['xmit_time'])
        df['arr_time'] = pd.to_datetime(df['arr_time'])
        df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
        df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
        df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
        df['lost'] = df['lost'].astype('boolean')
        df['excl'] = df['excl'].astype('boolean')
        df['latency'] = df['latency'].astype('float32')
    # E-UTRA & NR
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    if mode == 'eutra':
        df = df.rename(columns={
            'RSRP(dBm)': 'RSRP',
            'RSRQ(dB)': 'RSRQ',
            'Serving Cell Index': 'serv_cel_index',
            'Number of Neighbor Cells': 'num_neigh_cels',
            'Number of Detected Cells': 'num_cels',
            }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                        'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                        *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
        df.loc[df['serv_cel_index'] == '(MI)Unknown', 'serv_cel_index'] = '3_SCell'
        df['num_cels'] = df['num_neigh_cels'] + 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    if mode == 'nr':
        if df.empty:
            df = pd.DataFrame(columns=['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                                       'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0'])
        else:
            df = df.rename(columns={
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
                }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                            'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                            *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
            df.loc[df['serv_cel_pos'] != 255, 'serv_cel_index'] = 'PSCell'
            df[['PCI','RSRP','RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df.loc[df['serv_cel_pos'] == 255, 'num_neigh_cels'] = df['num_cels']
            df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    return df

In [63]:
root = "/Users/jackbedford/Desktop/MOXA/Code/data"
datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"

###
namelist = sorted([s for s in os.listdir(root) if s != '.DS_Store' and os.path.isdir(os.path.join(root, s))], reverse=False)
for i, s in enumerate(namelist):
    print(i, s)
print("Please select a date!")
date = namelist[int(input())]
print(date)
print()
datadir = os.path.join(root, date)

###
namelist = sorted([s for s in os.listdir(os.path.join(root, date)) if s != '.DS_Store' and os.path.isdir(os.path.join(root, date, s))], reverse=False)
for i, s in enumerate(namelist):
    print(i, s)
print("Please select an experiment!")
expr = namelist[int(input())]
print(expr)
print()
datadir = os.path.join(root, date, expr)

###
namelist = sorted([s for s in os.listdir(os.path.join(root, date, expr)) if s != '.DS_Store' and os.path.isdir(os.path.join(root, date, expr, s))], reverse=False)
for i, s in enumerate(namelist):
    print(i, s)
print("Please select a device!")
dev = namelist[int(input())]
print(dev)
print()
datadir = os.path.join(root, date, expr, dev)

###
namelist = sorted([s for s in os.listdir(os.path.join(root, date, expr, dev)) if s != '.DS_Store' and os.path.isdir(os.path.join(root, date, expr, dev, s))], reverse=False)
for i, s in enumerate(namelist):
    print(i, s)
print("Please select a trace!")
trace = namelist[int(input())]
print(trace)
print()
datadir = os.path.join(root, date, expr, dev, trace)

###
print(datadir, os.path.isdir(datadir))

0 2023-03-15
1 2023-03-16
2 2023-03-26
3 2023-04-01
4 2023-04-10
5 2023-04-17
6 2023-05-04
7 2023-05-07
8 2023-05-15
Please select a date!
2023-05-15

0 _Bandlock_8_Schemes_Phone
Please select an experiment!
_Bandlock_8_Schemes_Phone

0 sm00
1 sm01
2 sm02
3 sm03
4 sm04
5 sm05
6 sm06
7 sm07
Please select a device!
sm00

0 #01
1 #02
2 #03
3 #04
Please select a trace!
#02

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/_Bandlock_8_Schemes_Phone/sm00/#02 True


In [64]:
data = os.path.join(datadir, 'data', 'udp_uplk_loss_latency.csv')
print(data, os.path.isfile(data))
df_ul = pd.read_csv(data)
df_ul = set_data(df_ul)

data = os.path.join(datadir, 'data', 'udp_dnlk_loss_latency.csv')
print(data, os.path.isfile(data))
df_dl = pd.read_csv(data)
df_dl = set_data(df_dl)

data = os.path.join(datadir, 'data', [s for s in os.listdir(os.path.join(datadir, 'data')) if s.startswith('diag_log') and s.endswith('_ml1.csv') and not s.endswith('_nr_ml1.csv')][0])
print(data, os.path.isfile(data))
df_eutra = pd.read_csv(data)
df_eutra = set_data(df_eutra, mode='eutra', tz=8)
            
data = os.path.join(datadir, 'data', [s for s in os.listdir(os.path.join(datadir, 'data')) if s.startswith('diag_log') and s.endswith('_nr_ml1.csv')][0])
print(data, os.path.isfile(data))
df_nr = pd.read_csv(data)
df_nr = set_data(df_nr, mode='nr', tz=8)

data = os.path.join(datadir, 'data', [s for s in os.listdir(os.path.join(datadir, 'data')) if s.startswith('diag_log') and s.endswith('_rrc.csv')][0])
print(data, os.path.isfile(data))
df_ho = pd.read_csv(data)
df_ho, _, _ = mi_parse_ho(df_ho, tz=8)
df_ho['ho_type0'] = df_ho['ho_type'].astype('string')
df_ho.loc[df_ho['cause'].notna(), 'ho_type0'] = df_ho['ho_type'].astype('string') + '_' + df_ho['cause'].astype('string')
df_ho['ho_type0'] = df_ho['ho_type0'].astype('category')

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/_Bandlock_8_Schemes_Phone/sm00/#02/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/_Bandlock_8_Schemes_Phone/sm00/#02/data/udp_dnlk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/_Bandlock_8_Schemes_Phone/sm00/#02/data/diag_log_sm00_2023-05-15_15-19-16_ml1.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/_Bandlock_8_Schemes_Phone/sm00/#02/data/diag_log_sm00_2023-05-15_15-19-16_nr_ml1.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/_Bandlock_8_Schemes_Phone/sm00/#02/data/diag_log_sm00_2023-05-15_15-19-16_rrc.csv True


In [97]:
df_pcel = df_eutra.loc[df_eutra['serv_cel_index'] == 'PCell', ['Timestamp','PCI','EARFCN','RSRP','RSRQ']].reset_index(drop=True).copy()
df_pcel['Time'] = df_pcel['Timestamp'].dt.strftime("%H:%M:%S")
df_pcel['index_cng'] = df_pcel['PCI'].diff().fillna(1)
df_pcel['earfcn_cng'] = df_pcel['EARFCN'].diff().fillna(1)
table = df_pcel.loc[(df_pcel['index_cng'] != 0) | (df_pcel['earfcn_cng'] != 0), ['Time','PCI','EARFCN']].reset_index(drop=True).copy()
table.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Time,15:19:13,15:19:47,15:20:11,15:20:14,15:20:19,15:21:07,15:21:09,15:21:12,15:21:24,15:21:38,15:22:34,15:22:39,15:23:01,15:23:24,15:23:43
PCI,378,362,73,370,73,294,73,16,180,188,197,205,11,130,122
EARFCN,3050,3050,3050,3050,1750,1750,1750,1750,1750,1750,1750,1750,1750,1750,1750
